<a href="https://colab.research.google.com/github/AmelxJamal/NLP_Week1_Labs/blob/main/Amel_Abdelraheem_n_gram_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [1]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [4]:

print("load training set..")
print("\n")
train_data, vocab = load_data("/content/drive/MyDrive/NLP_Week1_Lab3/train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("/content/drive/MyDrive/NLP_Week1_Lab3/valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [5]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []

    ## FILL CODE
    for sentence in data:
      sentence_lst = []
      for word in sentence:
        if word not in vocab.keys():
          sentence_lst.append('unk')
        elif vocab[word] <= mincount:
          sentence_lst.append('unk')
        else:
          sentence_lst.append(word)
      data_with_unk.append(sentence_lst)
    
    return data_with_unk

In [6]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])


remove rare words
['<s>', 'my', 'unk', "don't", 'speak', 'unk', '</s>']


In [7]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    proba (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))
    context_counts = defaultdict(lambda: 0.0)

    for sentence in data:
        sentence = tuple(sentence)
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value
        for idx, word in enumerate(sentence):
          for gram in range(n): 
            if idx-gram >= 0:
              counts[sentence[idx-gram:idx]][word] +=1
              context_counts[sentence[idx-gram:idx]] +=1
            
    prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!

    for context in counts.keys():
    ## FILL CODE    
      for word in counts[context].keys():
        prob[context][word] = counts[context][word]/context_counts[context]
        
    return prob

In [64]:
# RUN TO BUILD NGRAM MODEL
n = 3
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

build ngram model with n =  3


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [85]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Returns:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    
    ## FILL CODE

    if context in model:
      if w in model[context]:
        return model[context][w]
      else:
        return 0.4 * get_prob(model, context[1:], w)
    else: # get_prob(model, context[1:], w) != None:
        return 0.4 * get_prob(model, context[1:], w)




In [86]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    ## FILL CODE
    perp, T = 0.0, 0
    for sentence in data:  
      sentence = tuple(sentence)  
      for idx,word in enumerate(sentence):
        if idx > 0:
          T+=1
          if n-1 < idx:
            perp -= np.log(get_prob(model,sentence[idx-n+1:idx], sentence[idx]))
          else: 
            perp -= np.log(get_prob(model,sentence[0:idx], sentence[idx]))
 
          

    return np.exp(perp/T)
    

In [87]:
# COMPUTE PERPLEXITY ON VALIDATION SET
print("The perplexity is", perplexity(model, valid_data, n=n))

The perplexity is 62.065880784180806


In [88]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    their probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE
    word_and_probs = defaultdict(lambda: 0.0)
    if context in model.keys():
      tot = 0
      for word in model[context].keys():
        word_and_probs[word] = model[context][word]
        tot += model[context][word]
      word_and_probs[word] = word_and_probs[word]/tot
    else:
      return get_proba_distrib(model, context[1:])
    
    return word_and_probs

In [89]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    n = 2 
    while sentence[-1] != "</s>" and len(sentence)<100:
        ## FILL CODE
        s_tuple = tuple(sentence)
        if n-len(sentence)<= 0:
          context = s_tuple
        else:
          context = s_tuple[-n:]
        words_dic = get_proba_distrib(model, context)
        word = max(words_dic.values())
        sentence.append(list(words_dic.keys())[list(words_dic.values()).index(word)])
        
    return sentence

In [90]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'tom', 'and', 'mary', 'said', 'they', 'were', 'unk', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.